In [ ]:
!ngrok config add-authtoken # Add your authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install langchain langchain-community openai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"]= # Add your key

In [ ]:

import pandas as pd
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings


# 1. اقرأ ملف Excel
df = pd.read_excel("Alinma BankV2.xlsx")

# 2. لكل صف، دمج الأعمدة في نص واحد
docs = []
for _, row in df.iterrows():
    combined_text = (
        f"القسم: {row['section']}\n"
        f"العنوان: {row['title']}\n"
        f"المحتوى: {row['content']}\n"
        f"الرابط: {row['url']}"
    )
    docs.append(Document(page_content=combined_text))

# 3. جهز الـ embeddings
embeddings = OpenAIEmbeddings()

# 4. أنشئ قاعدة FAISS
vector_store = FAISS.from_documents(docs, embeddings)

# 5. احفظ الفهرس
vector_store.save_local("faiss_index")

print("✅ تم إنشاء قاعدة بيانات FAISS وحفظها بنجاح")


/tmp/ipython-input-889076658.py:22: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


✅ تم إنشاء قاعدة بيانات FAISS وحفظها بنجاح


In [20]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
import json, re
from pyngrok import ngrok


# LangChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# API Key
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ngrok
public_url = ngrok.connect(8501)
print(f"🌐 افتح واجهتك من هنا: {public_url}")

with open("app.py", "w", encoding="utf-8") as f:
    f.write("""
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
import json, re
import os

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

if "main_page" not in st.session_state:
    st.session_state.main_page = "home"
if "step" not in st.session_state:
    st.session_state.step = 1

if st.session_state.main_page != "home":
    if st.button("⬅️ رجوع"):
        st.session_state.main_page = "home"
        st.session_state.step = 1
        st.rerun()

if st.session_state.main_page == "home":
    st.title("🏦 بنك الإنماء")
    st.write("اختر الخدمة التي تريدها:")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("💬 مساعد بنك الإنماء", use_container_width=True):
            st.session_state.main_page = "chatbot"
            st.rerun()
    with col2:
        if st.button("📈 خطتك الاستثمارية", use_container_width=True):
            st.session_state.main_page = "investment"
            st.rerun()

elif st.session_state.main_page == "investment":
    uploaded_file = st.sidebar.file_uploader("📂 ارفع كشف الحساب (CSV)", type="csv")
    if uploaded_file:
        df = pd.read_csv(uploaded_file)
        df["التاريخ"] = pd.to_datetime(df["التاريخ"])
        df["سنة-شهر"] = df["التاريخ"].dt.to_period("M")

        salary_data = df[df["التصنيف"] == "راتب شهري"]
        avg_salary = salary_data["المبلغ"].mean()
        expenses_data = df[df["النوع"] == "سحب"]
        monthly_expenses = expenses_data.groupby("سنة-شهر")["المبلغ"].sum().abs()
        avg_expenses = monthly_expenses.mean()
        surplus = avg_salary - avg_expenses

        if st.session_state.step == 1:
            st.title("📊 تحليل إنفاقك")
            col1, col2 = st.columns(2)
            with col1:
                plt.figure()
                plt.pie([avg_salary, avg_expenses], labels=["المتبقي", " المصروفات"], autopct='%1.1f%%')
                st.pyplot(plt)
            with col2:
                expenses_by_category = expenses_data.groupby("التصنيف")["المبلغ"].sum().abs()
                plt.figure()
                plt.pie(expenses_by_category, labels=expenses_by_category.index, autopct='%1.1f%%')
                st.pyplot(plt)

            st.write(f"✅ متوسط الدخل: {avg_salary:,.0f} ريال")
            st.write(f"💸 متوسط الصرف: {avg_expenses:,.0f} ريال")
            st.write(f"📌 الفائض: {surplus:,.0f} ريال")

            if st.button("التالي ➡️"):
                st.session_state.page_data = {
                    "الراتب": avg_salary,
                    "الصرف": avg_expenses,
                    "الفائض": surplus
                }
                st.session_state.step = 2
                st.rerun()

        elif st.session_state.step == 2:
            st.title("📈 الخطة المقترحة")
            plan_data = st.session_state.page_data
            proposed_investment = round(plan_data["الفائض"] * 0.7, 2)

            # 🔹 طلب القيم من الذكاء
            system_prompt = \"\"\"
أنت مستشار مالي في بنك الإنماء.
أعد خطة استثمارية بناءً على بيانات المستخدم.
الرد يجب أن يكون بصيغة JSON يحتوي:
- "النسبة": نسبة الاقتطاع من كل عملية شراء (عدد صحيح)
- "الخطورة": مستوى الخطورة (منخفض, متوسط, مرتفع)
\"\"\"

            user_prompt = f\"\"\"
بيانات المستخدم:
- متوسط الراتب الشهري: {plan_data["الراتب"]:.2f} ريال
- متوسط المصروفات الشهرية: {plan_data["الصرف"]:.2f} ريال
- الفائض الشهري: {plan_data["الفائض"]:.2f} ريال
- المبلغ المقترح للاستثمار (70% من الفائض): {proposed_investment:.2f} ريال
\"\"\"

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )

            raw_reply = response.choices[0].message.content
            cleaned_reply = re.sub(r"```json|```", "", raw_reply).strip()

            try:
                ai_plan = json.loads(cleaned_reply)
                base_percent = int(ai_plan.get("النسبة", 10))
                base_risk = ai_plan.get("الخطورة", "متوسط")
            except Exception:
                base_percent = 10
                base_risk = "متوسط"

            # نسخة ثابتة للنص
            fixed_invested_amount = proposed_investment
            fixed_percent = base_percent
            fixed_risk = base_risk

            # النص الثابت أول شيء
            st.info(f\"\"\"
بعد تحليل إنفاقك، توصّلنا إلى أن الاستثمار بعوائد {fixed_risk} المخاطر هو الأنسب لك.
سيتم استقطاع {fixed_invested_amount:,.2f} ريال شهريًا، من خلال اقتطاع نسبة {fixed_percent}% من كل عملية شراء تقوم بها، كضريبة شخصية ذكية.
ما يتبقى من المبلغ يتم استقطاعه تلقائيًا بتاريخ 27 من كل شهر.
الخطة قابلة للتعديل في أي وقت.
\"\"\")

            # مدخلات قابلة للتعديل (نسخة ثانية)
            invested_amount = st.number_input(
                "💵 المبلغ المستثمر شهريًا",
                min_value=0.0,
                value=fixed_invested_amount,
                step=100.0,
                format="%.2f"
            )

            percent_each_purchase = st.slider(
                "💳 النسبة المقتطعة من كل عملية شراء (%)",
                min_value=1,
                max_value=100,
                value=fixed_percent
            )

            risk_level = st.selectbox(
                "📊 مستوى الخطورة",
                ["منخفض", "متوسط", "مرتفع"],
                index=["منخفض", "متوسط", "مرتفع"].index(fixed_risk)
            )

            emergency_saving = int(max(0, plan_data["الفائض"] - invested_amount))
            st.write(f"🛡️ **مبلغ الطوارئ:** {emergency_saving} ريال")

            if st.button("➡️ التالي"):
                st.session_state.final_plan = {
                    "الراتب": plan_data["الراتب"],
                    "الصرف": plan_data["الصرف"],
                    "الفائض": plan_data["الفائض"],
                    "المبلغ_المستثمر": invested_amount,
                    "المبلغ_للطوارئ": emergency_saving,
                    "النسبة_المقتطعة": percent_each_purchase,
                    "مستوى_المخاطرة": risk_level
                }
                st.session_state.step = 3
                st.rerun()

        elif st.session_state.step == 3:
            st.title("🏦 محفظتك جاهزة!")

            funds_file = "/content/Funds Data.xlsx"
            try:
                funds_df = pd.read_excel(funds_file)
            except FileNotFoundError:
                st.error("⚠️ ملف الصناديق غير موجود. تأكد من رفع 'Funds Data.xlsx'")
                st.stop()

            system_prompt = \"\"\"
أنت خبير صناديق استثمار في بنك الإنماء.
اختر الصندوق الأنسب بناءً على مستوى المخاطرة والمبلغ المستثمر.
أعط الرد بصيغة JSON يحتوي:
- "اسم الصندوق"
- "نوع الصندوق"
- "مستوى المخاطرة"
- "العائد المتوقع سنويًا (%)"
- "الرسوم السنوية"
- "مدة الاستثمار المقترحة"
- "إجمالي المبلغ المتوقع بنهاية المدة"
- "النبذة": شرح قصير بالعربية
\"\"\"

            user_prompt = f\"\"\"
بيانات المستخدم:
- المبلغ المستثمر: {st.session_state.final_plan["المبلغ_المستثمر"]}
- مستوى المخاطرة: {st.session_state.final_plan["مستوى_المخاطرة"]}
- النسبة المقتطعة من المشتريات: {st.session_state.final_plan["النسبة_المقتطعة"]}%
- المبلغ للطوارئ: {st.session_state.final_plan["المبلغ_للطوارئ"]}

بيانات الصناديق:
{funds_df.to_dict(orient="records")}
\"\"\"

            if st.button("📌 احصل على الصندوق المقترح"):
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ]
                )
                raw_reply = response.choices[0].message.content
                cleaned_reply = re.sub(r"```json|```", "", raw_reply).strip()

                try:
                    fund = json.loads(cleaned_reply)
                    st.session_state.ai_fund = fund
                    st.success("✅ تم اقتراح الصندوق بنجاح")
                except json.JSONDecodeError:
                    st.error("⚠️ تعذر قراءة JSON من الرد")

            if "ai_fund" in st.session_state:
                fund = st.session_state.ai_fund
                st.subheader("📖 نبذة عن الصندوق:")
                st.info(fund.get("النبذة", "لا توجد نبذة متاحة"))

                st.subheader("📊 تفاصيل الصندوق:")
                for key, value in fund.items():
                    if key != "النبذة":
                        st.write(f"**{key}:** {value}")

                if st.button("✅ إنشاء المحفظة"):
                    st.success("🎉 تم إنشاء محفظتك الاستثمارية بنجاح!")
""")

!streamlit run app.py --server.port 8501 --server.address 0.0.0.0


🌐 افتح واجهتك من هنا: NgrokTunnel: "https://261deba82e54.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

/content/app.py:11: LangChainDeprecationWarning: Importing chat models from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.chat_models import ChatOpenAI`.

To install langchain-community run `pip install -U langchain-community`.
  from langchain.chat_models import ChatOpenAI
/content/app.py:11: LangChainDeprecationWarning: Importing chat models from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.chat_models import ChatOpenAI`.

To install langchain-community run `pip install -U langchain-community`.
  from langchain.chat_models import ChatOpenAI
/